In [ ]:
!pip install gdown

In [ ]:
!gdown --id 18M9NIQAPoV7Hq_1csWDizMLOye80NgvL

In [ ]:
import os
# os.system(f"rm -rf /kaggle/working/OutputForRobertaZeroShotFinal.csv")

In [ ]:
import os

# Get the current working directory
cwd = os.getcwd()

# Get the path to the folder to unzip
folder_path = "/kaggle/working/roberta67k50n.zip"

# Create a new directory to unzip the folder into
new_folder_path = os.path.join(cwd, "urduHack67k50n")

# Unzip the folder
os.system(f"unzip -q {folder_path} -d {new_folder_path}")

# Remove the original folder
os.system(f"rm -rf {folder_path}")

In [ ]:
# from transformers import BertTokenizer, RobertaTokenizer, EncoderDecoderModel,AutoTokenizer
# import transformers
# model = EncoderDecoderModel.from_pretrained("/kaggle/working/roberta2robertaxlm67k50n")
# # tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
# tokenizer = AutoTokenizer.from_pretrained("urduhack/roberta-urdu-small")

In [1]:
!pip install nlp

In [2]:
from transformers import  EncoderDecoderModel,AutoTokenizer
model = EncoderDecoderModel.from_pretrained("/kaggle/working/roberta2robertaxlm67k50n147meet")
tokenizer =  AutoTokenizer.from_pretrained("urduhack/roberta-urdu-small")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
import pandas as pd

In [4]:
dataset_file1 = '/kaggle/input/60meetings/urdu_train.csv'
dataset_file2 = '/kaggle/input/60meetings/urdu_val.csv'
dataset1 = pd.read_csv(dataset_file1)
dataset2 = pd.read_csv(dataset_file2)

In [ ]:
# dataset1=dataset1[:30000]
# dataset2=dataset2[:1000]

In [5]:
from datasets import load_dataset,concatenate_datasets,Dataset

In [6]:
train_dataset = pd.DataFrame()
train_dataset['article'] =dataset1['text']
train_dataset['highlights'] =dataset1['summary']
train_dataset = Dataset.from_pandas(train_dataset)
train_dataset

Dataset({
    features: ['article', 'highlights'],
    num_rows: 50
})

In [7]:
val_dataset = pd.DataFrame()

val_dataset['article'] ='Text :'+dataset2['text']
val_dataset['highlights'] ='Summary :'+dataset2['summary']
val_dataset = Dataset.from_pandas(val_dataset)
val_dataset

Dataset({
    features: ['article', 'highlights'],
    num_rows: 10
})

In [8]:
import nlp

In [9]:
!pip install rouge_score rouge_score

In [10]:
# load rouge for validation
rouge = nlp.load_metric("rouge", experiment_id=0)

# set decoding params
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.config.max_length = 142
model.config.min_length = 56
model.config.no_repeat_ngram_size = 3
model.early_stopping = True
model.length_penalty = 2.0
model.num_beams = 4

encoder_length = 512
decoder_length = 512
batch_size = 1


In [11]:

# map data correctly
def map_to_encoder_decoder_inputs(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    # cut off at Longformer at 2048
    inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=encoder_length)
    # force summarization <= 256
    outputs = tokenizer(batch["highlights"], padding="max_length", truncation=True, max_length=decoder_length)

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_input_ids"] = outputs.input_ids
    batch["labels"] = outputs.input_ids.copy()
    # mask loss for padding
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]
    ]
    batch["decoder_attention_mask"] = outputs.attention_mask

    assert all([len(x) == encoder_length for x in inputs.input_ids])
    assert all([len(x) == decoder_length for x in outputs.input_ids])

    return batch


def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.eos_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }



In [12]:
# make train dataset ready
train_dataset = train_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=batch_size, remove_columns=["article", "highlights"],
)
train_dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_attention_mask", "decoder_input_ids", "labels"],
)

# same for validation dataset
val_dataset = val_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=batch_size, remove_columns=["article", "highlights"],
)
val_dataset.set_format(
    type="torch", columns=["input_ids", "decoder_attention_mask", "attention_mask", "decoder_input_ids", "labels"],
)

  0%|          | 0/50 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [13]:
from transformers import TrainingArguments, Trainer,EncoderDecoderModel,AutoTokenizer

In [14]:

# set training arguments - these params are not really tuned, feel free to change
training_args = TrainingArguments(
    output_dir="./",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    do_train=True,
    do_eval=False,
    logging_steps=1000,
    save_steps=1000,
    eval_steps=1000,
    overwrite_output_dir=True,
    warmup_steps=2000,
    save_total_limit=3,
    fp16=True,
    num_train_epochs=10,
)

# instantiate trainer
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [15]:
# start training
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Currently logged in as: mahmd220339. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.10/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss


TrainOutput(global_step=250, training_loss=0.13683909606933595, metrics={'train_runtime': 210.9947, 'train_samples_per_second': 2.37, 'train_steps_per_second': 1.185, 'total_flos': 306760040448000.0, 'train_loss': 0.13683909606933595, 'epoch': 10.0})

In [16]:
trainer.save_model("roberta2robertaxlm67k50n147meet60meet")

In [17]:
import shutil
shutil.make_archive('UrduHackroberta67k50n147s60m', 'zip', '/kaggle/working/roberta2robertaxlm67k50n147meet60meet')

'/kaggle/working/UrduHackroberta67k50n147s60m.zip'

In [ ]:
# from transformers import EncoderDecoderModel, AutoTokenizer

In [ ]:
# model_ckpt = "xlm-roberta-base"
# tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
# model = EncoderDecoderModel.from_pretrained("/kaggle/working/roberta2robertaxlm")

In [ ]:
# dataset_test=pd.read_csv('/kaggle/input/xlsum-csv/urdu_test.csv')
# prompt=dataset_test['Question'][10]
# print(prompt)

In [ ]:
# inputs = tokenizer(prompt, padding="max_length", truncation=True, max_length=512, return_tensors="pt")

In [ ]:
# outputs = model.generate(inputs.input_ids, attention_mask=inputs.attention_mask)

In [ ]:
# output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [ ]:
# output_str

# For Zero shot

In [ ]:
# from transformers import   EncoderDecoderModel,AutoTokenizer
# model = EncoderDecoderModel.from_pretrained("patrickvonplaten/roberta2roberta-cnn_dailymail-fp16")
# # tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
# tokenizer = AutoTokenizer.from_pretrained("urduhack/roberta-urdu-small")

In [ ]:
from transformers import   EncoderDecoderModel,AutoTokenizer
model = EncoderDecoderModel.from_pretrained("/kaggle/working/urduHack67k50n/roberta67k50n").to('cuda:0').half()
tokenizer =  AutoTokenizer.from_pretrained("urduhack/roberta-urdu-small")

In [ ]:
import pandas as pd

In [ ]:
dataset_test=pd.read_csv('/kaggle/input/meetings1-10/meeting1.csv')
prompt=dataset_test['subpart'][0]
print(prompt)

In [ ]:
inputs = tokenizer(prompt, padding="max_length", truncation=True, max_length=512, return_tensors="pt")

In [ ]:
input_ids = inputs.input_ids.to('cuda:0')
attention_mask = inputs.attention_mask.to('cuda:0')

In [ ]:
outputs = model.generate(input_ids, attention_mask=attention_mask,max_length=512)

In [ ]:
output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [ ]:
output_str

In [ ]:
crows_pairs=pd.read_csv('/kaggle/input/xlsum-csv/urdu_test.csv')

In [ ]:
csv_file_path = '/kaggle/input/testsamples/sample1.csv'
df = pd.read_csv(csv_file_path)
for n in range(1751,3000):
    data=crows_pairs['Question'][n]
    inputs = tokenizer(data, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to('cuda:0')
    attention_mask = inputs.attention_mask.to('cuda:0')
    outputs = model.generate(input_ids, attention_mask=attention_mask,max_length=512)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
#     print(output_str)
#     while len(result=='') == 0:
#             time.sleep(0.1)
    new_record_data = {
    'Original': crows_pairs['Answer'][n],
    'Generated': str(output_str),
    }
    df = df.append(new_record_data, ignore_index=True)
    output_csv_file_path = '/kaggle/working/Roberta67k50nNews8500From1751to2999.csv'
    df.to_csv(output_csv_file_path, index=False)
    print(n)

In [ ]:
originalmeet=pd.read_csv('/kaggle/input/10meetings/test_data_10meetings.csv')

In [ ]:
csv_file_path = '/kaggle/input/testsamples/sample1.csv'
df = pd.read_csv(csv_file_path)
for m in range (1,11):
    filename='/kaggle/input/meetings1-10/meeting'+str(m)+'.csv'
    summary=''
    dataset_test=pd.read_csv(filename)
    for n in range(dataset_test.shape[0]):
        text=dataset_test['subpart'][n]
        inputs = tokenizer(text, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
        input_ids = inputs.input_ids.to('cuda:0')
        attention_mask = inputs.attention_mask.to('cuda:0')
        outputs = model.generate(input_ids, attention_mask=attention_mask,max_length=512)
        output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        summary=summary+str(output_str)
        print(output_str)
    original=originalmeet['summary'][m-1]
    new_record_data = {
    'Original': original,
    'Generated': summary,
    }
    df = df.append(new_record_data, ignore_index=True)
    output_csv_file_path = '/kaggle/working/OutputForRoberta67k50n10Meet.csv'
    df.to_csv(output_csv_file_path, index=False)

In [ ]:
samples=pd.read_csv('/kaggle/working/OutputForRoberta67k50n10Meet.csv')

In [ ]:
samples.shape

In [ ]:
!pip install rouge

In [ ]:
from rouge import Rouge

In [ ]:
def calc_rouge(reference,hypothesis):
  rouge = Rouge()
  scores = rouge.get_scores(hypothesis, reference)
  return scores[0]['rouge-1']['f'],scores[0]['rouge-2']['f'],scores[0]['rouge-l']['f']

In [ ]:
import csv,numpy as np
with open('/kaggle/working/OutputForRoberta67k50n10Meet.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    rouge_1_scores = []
    rouge_2_scores = []
    rouge_l_scores = []
    rouge_lsum_scores = []
    for row in reader:
        original_summary = row[0]
        generated_summary = row[1]
        rouge_1_score,rouge_2_score,rouge_l_score = calc_rouge(original_summary, generated_summary)
        rouge_1_scores.append(rouge_1_score)
        rouge_2_scores.append(rouge_2_score)
        rouge_l_scores.append(rouge_l_score)
    print('Rouge-1 score:', np.mean(rouge_1_scores))
    print('Rouge-2 score:', np.mean(rouge_2_scores))
    print('Rouge-L score:', np.mean(rouge_l_scores))

In [ ]:
import sys

In [ ]:
sys.setrecursionlimit(25000)